> Scalability cannot be tested on Google Colab due to its limited resources. 

> We tested scalabilty on an AMD 16-core processor, with 128 GB RAM and an NVIDIA Tesla T4 GPU.

# Running Code

In [ ]:
import logging
import matplotlib.pyplot as plt
import time
from dotmap import DotMap
from pytorch_lightning import seed_everything
from torch.nn.init import kaiming_uniform_ 
from utils.faulty_client_localization.FaultyClientLocalization import FaultyClientLocalization
from utils.faulty_client_localization.InferenceGuidedInputs import InferenceGuidedInputs
from utils.FLSimulation import trainFLMain
from utils.fl_datasets import initializeTrainAndValidationDataset
from utils.util import aggToUpdateGlobalModel
from utils.util import testAccModel



logging.basicConfig(filename='example.log', level=logging.ERROR)
logger = logging.getLogger("pytorch_lightning")
seed_everything(786)



def evaluateFaultLocalization(predicted_faulty_clients_on_each_input, true_faulty_clients):
    true_faulty_clients = set(true_faulty_clients)
    detection_acc = 0
    for pred_faulty_clients in predicted_faulty_clients_on_each_input:
        print(f"+++ Faulty Clients {pred_faulty_clients}")
        correct_localize_faults = len(
            true_faulty_clients.intersection(pred_faulty_clients))
        acc = (correct_localize_faults/len(true_faulty_clients))*100
        detection_acc += acc
    fault_localization_acc = detection_acc / \
        len(predicted_faulty_clients_on_each_input)
    return fault_localization_acc


def runFaultyClientLocalization(client2models, exp2info, num_bugs, random_generator=kaiming_uniform_, apply_transform=True, k_gen_inputs=10, na_threshold=0.003, use_gpu=True):
    print(">  Running FaultyClientLocalization ..")
    input_shape = list(exp2info['data_config']['single_input_shape'])
    generate_inputs = InferenceGuidedInputs(client2models, input_shape, randomGenerator=random_generator, apply_transform=apply_transform,
                                            dname=exp2info['data_config']['name'], min_nclients_same_pred=5, k_gen_inputs=k_gen_inputs)
    selected_inputs, input_gen_time = generate_inputs.getInputs()

    start = time.time()
    faultyclientlocalization = FaultyClientLocalization(
        client2models, selected_inputs, use_gpu=use_gpu)

    potential_benign_clients_for_each_input = faultyclientlocalization.runFaultLocalization(
        na_threshold, num_bugs=num_bugs)
    fault_localization_time = time.time()-start
    return potential_benign_clients_for_each_input, input_gen_time, fault_localization_time





# ====== Simulation ===== 

args = DotMap()
args.lr = 0.001
args.weight_decay = 0.0001
args.batch_size = 512
args.faulty_clients_ids = "0,1" # can be multiple clients separated by comma e.g. "0,1,2"  but keep under args.clients clients and at max less than 7 
args.sampling = "iid" # [iid, "niid"] 
args.noise_rate = 1  # noise rate 0 to 1 


### Figure 9: Densenet and Femnist

In [ ]:
args.dataset = "femnist" # ['cifar10', 'femnist']
args.model = "densenet121" # [resnet18, resnet34, resnet50, densenet121, vgg16]
args.epochs = 1  # range 10-25


# different_num_clients = [3, 5, 10] # keep under 100
different_num_clients = [25, 50, 100, 200, 400] # keep under 400
numclients2acc = []
for clients in different_num_clients:
    args.clients = clients  
    c2ms, exp2info = trainFLMain(args)
    client2models = {k: v.model.eval() for k, v in c2ms.items()}
    potential_faulty_clients, _, _ = runFaultyClientLocalization(
        client2models=client2models, exp2info=exp2info, num_bugs=len(exp2info['faulty_clients_ids']))

    fault_acc = evaluateFaultLocalization(
        potential_faulty_clients, exp2info['faulty_clients_ids'])


    print(f"#Figure 9: Fault Localization Accuracy: {fault_acc}, Distribution: {args.sampling},  Faulty clients: {len(args.faulty_clients_ids.split(','))}, Total Clients: {args.clients}, Architecture: {args.model}, Dataset: {args.dataset}")
    numclients2acc.append(fault_acc)



x = different_num_clients 

fig, axes = plt.subplots(1, 1) 
axes.plot(x, numclients2acc, 'g--o') 
# axes[1].plot(x, y2_fault_acc, 'm--o')

axes.set_xlabel('Clients')
axes.set_ylabel('Fault Localization Accuracy (%)')
axes.set_title("Paper Figure-9 (a)")


axes.set_ylim(0, 105)


fig.set_figwidth(5)
plt.tight_layout()
